In [1]:
import dotenv
dotenv.load_dotenv()

True

### Build Agent with Json action format

We have learned how agents works and how to build them in LangChain from scratch.

But in practise, what you want to do is to take full advantage of the framework and build your own agent with the least effort.
This is the excerise we are going to do in this section.

We will take a prompt from langchain hub which asks the model to output action in json format.

You task is to connect the components into a working agent.

In [2]:
import os

from langchain import hub
from langchain.chat_models import ChatOpenAI

### The Prompt

We will use one of popular prompt available on langchain hub `hwchase17/react-multi-input-json`

It consist of two messages:
- system one, which descripes the how the agent should response and available prompts
- human message which will contain current input and gent scratchpad with previously executed actions and their results

In [3]:
prompt = hub.pull("hwchase17/react-multi-input-json")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tool_names', 'tools'], template='Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\n{tools}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\nValid "action" values: "Final Answer" or {tool_names}\n\nProvide only ONE action per $JSON_BLOB, as shown:\n\n```\n{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}\n```\n\nFollow this format:\n\nQuestion: input question to answer\nThought: consider previous and subsequent steps\nAction:\n```\n$JSON_BLOB\n```\nObservation: action result\n... (repeat Thought/Action/Observation N times)\nThought: I know what to respond\nAction:\n```\n{{\n  "action": "Final Answer",\n  "action_input": "Final response to human"\n}}\n\nBegin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format

In [4]:
prompt = hub.pull("hwchase17/react-multi-input-json")

for message in prompt.messages:
    print(f"Input variables: f{message.prompt.input_variables}")
    print(f"Prompt:\n{message.prompt.template}")
    print("\n " + "="* 50 + "\n")

Input variables: f['tool_names', 'tools']
Prompt:
Respond to the human as helpfully and accurately as possible. You have access to the following tools:

{tools}

Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Valid "action" values: "Final Answer" or {tool_names}

Provide only ONE action per $JSON_BLOB, as shown:

```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```

Follow this format:

Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{{
  "action": "Final Answer",
  "action_input": "Final response to human"
}}

Begin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation


Input variables: f['agent_scratchpad', 'i

### Tools

Here you can define the tools you want to use in your agent.

If environment variables for Serep and Wolfram are set, we will automatically add those tools

In [5]:
from langchain.utilities import GoogleSerperAPIWrapper, DuckDuckGoSearchAPIWrapper, WolframAlphaAPIWrapper
from langchain.tools import Tool

tools = [

]

if os.getenv("SERPER_API_KEY") is None:
    print("Serper Api Key not set!")
else:
    tools.append(Tool(
        name="web_search",
        func=GoogleSerperAPIWrapper().run,
        description="Usefull if you need to find out some additional information. You should ask targeted questions."
    ),)
if os.getenv("WOLFRAM_ALPHA_APPID") is None:
    print("Wolfram Alpha App ID not set!")
else:
    tools.append(Tool(
        name="calculator", 
        func=WolframAlphaAPIWrapper().run,
        description="Calculator. Use it for any calculation. Input should be a string with equation."
    ),)

### Build Agent chain

Here we will test your understanding of LangChain building blocks and how Agents work.

Steps which you need to do:
- create `prompt_formatted`, check the system prompt and what input variables it has. Tools will be the same at each step, so you can either build chain which always format the prompt with them or use prompt function `.partial` which will partially format it
- create `llm_with_stop`, recall from the previous notebook, that we want to stop LLM from generating new tokens when it outputs observation step to execute the action and provide its results as observation
- create `agent` which will
    - take current input consisting of `input` and `intermediate_steps` and passes the input and somehow created `agent_scratchpad` to the prompt
    - formats the prompt with current `input` and `agent_scratchpad`
    - calls `llm_with_stop` to produce output
    - extracts the structured output in JSON using `JSONAgentOutputParser`

In [6]:
from langchain.agents import AgentExecutor

from langchain.tools.render import render_text_description_and_args # Render the tool name, description, and args in plain text.
from langchain.agents.format_scratchpad import format_log_to_str # Construct the scratchpad that lets the agent continue its thought process.
from langchain.agents.output_parsers import JSONAgentOutputParser # Parses tool invocations and final answers in JSON format.

prompt_formatted = prompt.partial(
    tools=render_text_description_and_args(tools),
    tool_names=", ".join([t.name for t in tools]),
)

llm = ChatOpenAI(model="gpt-3.5-turbo-0613", temperature=0)
llm_with_stop = llm.bind(stop=["Observation"])

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt_formatted
    | llm_with_stop
    | JSONAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, max_steps=3, verbose=True, handle_parsing_errors=True) # type: ignore

In [7]:
response = agent_executor.invoke(
    {"input": "What is the price of 123 Tesla stocks?"}
)




> Entering new AgentExecutor chain...
Thought: I need to find the current price of Tesla stocks.

Action:
```{
  "action": "web_search",
  "action_input": "current price of Tesla stocks"
}```234.30 +0.71 (0.30%)Action:
```{
  "action": "Final Answer",
  "action_input": "The current price of 123 Tesla stocks is $234.30 per share."
}```

> Finished chain.


In [8]:
response['output']

'The current price of 123 Tesla stocks is $234.30 per share.'

### Solution

In [9]:
from langchain.agents import AgentExecutor

from langchain.tools.render import render_text_description_and_args # Render the tool name, description, and args in plain text.
from langchain.agents.format_scratchpad import format_log_to_str # Construct the scratchpad that lets the agent continue its thought process.
from langchain.agents.output_parsers import JSONAgentOutputParser # Parses tool invocations and final answers in JSON format.

prompt_formatted = prompt.partial(
    tools=render_text_description_and_args(tools),
    tool_names=", ".join([t.name for t in tools]),
)

llm = ChatOpenAI(model="gpt-3.5-turbo-0613", temperature=0)
llm_with_stop = llm.bind(stop=["Observation"])

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt_formatted
    | llm_with_stop
    | JSONAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, max_steps=3, verbose=True, handle_parsing_errors=True) # type: ignore